# Covid-19 Dashboard

In [1]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.core.display import display, HTML
import ipywidgets as widgets
import pandas as pd
import numpy as np
import folium
import plotly.graph_objects as go

In [2]:
death_df = pd.read_csv("https://raw.githubusercontent.com/microsoft/Bing-COVID-19-Data/master/data/Bing-COVID19-Data.csv")

In [54]:
# qwer=death_df[death_df['Country_Region'] == 'Worldwide']
# qwer

In [4]:
# map_data=death_df.loc[death_df['Updated']=='06/20/2020', ['Latitude','Longitude','AdminRegion2','Confirmed']]
# map_data=map_data.dropna()
# map_data.head(1)


In [26]:
# map_data=death_df.loc[death_df['Updated']=='06/20/2020', ['Latitude','Longitude','AdminRegion2','Confirmed']]
# map_data=map_data.dropna()
# world_map = folium.Map(location=[20.5,78.9], tiles='openstreetmap', zoom_start=4, max_zoom= 6, min_zoom=2)

# for i in range(len(map_data)):
#     folium.Circle(
#         location=[map_data.iloc[i]['Latitude'], map_data.iloc[i]['Longitude']],
#         fill=False,
#         radius = (int((np.log(map_data.iloc[i]['Confirmed']+1.00001)))+0.2)*3500,
#         fill_color = 'blue',
#         color = 'red',
# #         tooltip = "<div style='margin: 0; background-color: powderblue; color: blue;'>"+
# #                     "<h4 style='text-align:center;font-weight: bold'>"+confirmed_df.iloc[i]['country'] + "</h4>"
# #                     "<hr style='margin:10px;color: white;'>"+
# #                     "<ul style='color: blue;;list-style-type:circle;align-item:left;padding-left:20px;padding-right:20px'>"+
# #                         "<li>Confirmed: "+str(confirmed_df.iloc[i,-1])+"</li>"+
# #                         "<li>Deaths:   "+str(death_df.iloc[i,-1])+"</li>"+
# #                         "<li>Death Rate: "+ str(np.round(death_df.iloc[i,-1]/(confirmed_df.iloc[i,-1]+1.00001)*100,2))+ "</li>"+
# #                     "</ul></div>",
#         ).add_to(world_map)

    
# world_map    

In [6]:
# Select your Country to get data:

In [7]:
# def country_data1(country):
#     data_country=death_df[(death_df['Country_Region'] == country)&(death_df['AdminRegion2'].isnull())&(death_df['AdminRegion1'].isnull())].tail(1)
#     display(HTML("<div style = 'background-color: #504e4e; padding: 30px '>" +
#              "<span style='color: yellow; font-size:30px;'> Confirmed: "  + str(int(data_country.iloc[0,2])) +"</span>" +
#              "<span style='color: red; font-size:30px;margin-left:20px;'> Deaths: " + str(int(data_country.iloc[0,4])) + "</span>"+
#              "<span style='color: lightgreen; font-size:30px; margin-left:20px;'> Recovered: " + str(int(data_country.iloc[0,6])) + "</span>"+
#              "</div>")
#                )
        

# list_country=list(death_df.Country_Region)
# list_country=list(dict.fromkeys(list_country))
# interact(country_data1, country=list_country);

In [8]:
# Select your State to get data:

In [9]:
# def state_data2(state):
#     data_state=death_df[(death_df['AdminRegion1'] == state)&(death_df['AdminRegion2'].isnull())].tail(1)
#     if data_state.empty:
#         print('No data found')
#     else:
#         display(HTML("<div style = 'background-color: #504e4e; padding: 30px '>" +
#              "<span style='color: yellow; font-size:30px;'> Confirmed: "  + str(data_state.iloc[0,2]) +"</span>" +
#              "<span style='color: red; font-size:30px;margin-left:20px;'> Deaths: " + str(data_state.iloc[0,4]) + "</span>"+
#              "<span style='color: lightgreen; font-size:30px; margin-left:20px;'> Recovered: " + str(data_state.iloc[0,6]) + "</span>"+
#              "</div>")
#                )


# def country_data2(country):
#     list_state=death_df[death_df['Country_Region'] == country]
#     list_state=list(list_state.AdminRegion1)
#     list_state=[x for x in list_state if str(x)!='nan']
#     #print(list_state)
#     if not list_state:
#         print("No state available")
#     else:    
#         list_state=list(dict.fromkeys(list_state))
#         #print(list_state)
#         #list_state=list_state[1:]
#         interact(state_data2, state=list_state);

# list_country=list(death_df.Country_Region)
# list_country=list(dict.fromkeys(list_country))
# interact(country_data2, country=list_country);

# Select your Country to get data:

In [134]:
list_date=death_df.loc[death_df['Country_Region'] == 'India',['Updated']].tail(3)
list_date=[x for x in list_date.iloc[:,0]]


In [88]:
def country_data3(Country):
    data_country=death_df[(death_df['Country_Region'] == Country)&(death_df['AdminRegion2'].isnull())&(death_df['AdminRegion1'].isnull())].tail(1)
    data_country.fillna(0, inplace=True)
    if data_country.empty:
        print('No data found')
    else:
        display(HTML("<div style = 'background-color: #504e4e; padding: 30px '>" +
             "<span style='color: yellow; font-size:30px;'> Confirmed: "  + str(int(data_country.iloc[0,2])) +"</span>" +
             "<span style='color: red; font-size:30px;margin-left:20px;'> Deaths: " + str(int(data_country.iloc[0,4])) + "</span>"+
             "<span style='color: lightgreen; font-size:30px; margin-left:20px;'> Recovered: " + str(int(data_country.iloc[0,6])) + "</span>"+
             "</div>")
               )


list_country=list(death_df.Country_Region)
list_country=list(dict.fromkeys(list_country))
interact(country_data3, Country=list_country);

interactive(children=(Dropdown(description='Country', options=('Worldwide', 'Afghanistan', 'Albania', 'Algeria…

# Select your State to get data:

In [90]:
def state_data3(State='Uttar Pradesh'):
    data_state=death_df[(death_df['AdminRegion1'] == State)&(death_df['AdminRegion2'].isnull())&((death_df['Updated'] == list_date[2])|(death_df['Updated'] == list_date[1])|(death_df['Updated'] == list_date[0]))].tail(1)
    data_state.fillna(0, inplace=True)
    if data_state.empty:
        print('No data found')
    else:
        display(HTML("<div style = 'background-color: #504e4e; padding: 30px '>" +
             "<span style='color: yellow; font-size:30px;'> Confirmed: "  + str(int(data_state.iloc[0,2])) +"</span>" +
             "<span style='color: red; font-size:30px;margin-left:20px;'> Deaths: " + str(int(data_state.iloc[0,4])) + "</span>"+
             "<span style='color: lightgreen; font-size:30px; margin-left:20px;'> Recovered: " + str(int(data_state.iloc[0,6])) + "</span>"+
             "</div>")
               )
list_state=list(death_df.AdminRegion1)
list_state=[x for x in list_state if str(x)!='nan']
#print(list_state)
list_state=list(dict.fromkeys(list_state))
#print(list_state)
#list_state=list_state[1:]
interact(state_data3, State=list_state);


interactive(children=(Dropdown(description='State', index=350, options=('Catamarca', 'Cordoba', 'Chaco', 'City…

# Select your City to get data:

In [123]:
def city_data3(city='Lucknow'):
    data_city=death_df[(death_df['AdminRegion2'] == city)&((death_df['Updated'] == list_date[2])|(death_df['Updated'] == list_date[1])|(death_df['Updated'] == list_date[0]))]
    data_city=data_city.tail(1)
    data_city.fillna(0, inplace=True)
    if data_city.empty:
        print('No data found')
    else:
        display(HTML("<div style = 'background-color: #504e4e; padding: 30px '>" +
             "<span style='color: yellow; font-size:30px;'> Confirmed: "  + str(int(data_city.iloc[0,2])) +"</span>" +
             "<span style='color: red; font-size:30px;margin-left:20px;'> Deaths: " + str(int(data_city.iloc[0,4])) + "</span>"+
             "<span style='color: lightgreen; font-size:30px; margin-left:20px;'> Recovered: " + str(int(data_city.iloc[0,6])) + "</span>"+
             "</div>")
               )
list_city=list(death_df.AdminRegion2)
list_city=[y for y in list_city if str(y)!='nan']
#print(list_state)
list_city=list(dict.fromkeys(list_city))
#list_city=list_city[1:]
interact(city_data3, city=list_city);    

interactive(children=(Dropdown(description='city', index=708, options=('Yukon', 'Easter Island', 'Alajuela', '…

In [137]:
# date=death_df.loc[death_df['Country_Region'] == 'Worldwide',['Updated']].tail(1)
# date=date.iloc[0,0]

In [143]:
# special=death_df.loc[death_df['Updated']=='06/20/2020', ['Latitude','Longitude','AdminRegion2','Confirmed']].dropna()
# special
# # map_data=death_df.loc[death_df['Updated']=='06/20/2020', ['Latitude','Longitude','AdminRegion2','Confirmed']]
# # map_data=map_data.dropna()

In [135]:
# date1='06/21/2020'
# if date1>date:
#     print('correct')
# else:
#     print('False')